### Collecting & processing data imports and functions:
Run this cell (select, then shift+enter)

In [1]:
#@title
from time import sleep
from tqdm import tqdm
import requests
import json
import pandas as pd


def make_deep_requests(url, depth=5, limit=1000, sleep_time=2, stop_id=None):

    last_id = None
    data = pd.DataFrame()

    for counter in tqdm(range(depth)):
        if last_id:
            url += f'&lastId={last_id}'

        try:
            response = requests.get(url)
            new_data = json_normalize(json.loads(response.text))
            data = data.append(new_data)

        except Exception as exception:
            print(exception)
            import pdb; pdb.set_trace()

        last_id = data.id.iloc[-1]

        if len(new_data) < limit:
            break

        if stop_id in data['id'].values:
            break

        time.sleep(sleep_time)

    return data


def response_tokens(address, network='mainnet', offset=None):
    url = f'https://api.better-call.dev/v1/contract/{network}/{address}/tokens'
    if offset:
        url += f'?offset={offset}'
    return requests.get(url)


def process_response(response):
    data = json.loads(response.text)
    return pd.DataFrame(data)


def response_tokens_deep(address, network='mainnet', max_requests=2, sleep_time=1):
    tokens = pd.DataFrame()
    offset_size = 10
    offset = 0

    for _ in tqdm(range(max_requests)):
        response = response_tokens(address, network, offset)
        more_tokens = process_response(response)
        if not len(more_tokens):
            break

        tokens = tokens.append(more_tokens)
        offset += 10
        sleep(sleep_time)

    return tokens


def collect_token_holders(address, token_id, decimals, network='mainnet'):

    url = f'https://api.better-call.dev/v1/contract/{network}/{address}/tokens/holders?token_id={token_id}'
    response = requests.get(url)
    data = json.loads(response.text)
    holders = pd.Series(data).astype(float) / (10**decimals)

    holders = pd.DataFrame({
        'total': holders,
        'share': holders / holders.sum()
    }).sort_values('share', ascending=False)
    holders['cum_share'] = holders.share.cumsum()
    holders['rank'] = pd.Series(list(range(len(holders))), index=holders.index) + 1

    return holders


def iterate_with_pause(iterable, pause=1):
    for item in iterable:
        yield item
        sleep(pause)


def collect_token_holders_data(tokens):
    token_holders = pd.concat({
        token.symbol: collect_token_holders(token.contract, token.token_id, token.decimals)
        for _, token in tqdm(iterate_with_pause(tokens.iterrows()))
    })

    return token_holders


def request_coins_list_from_coingecko():
    response = requests.get('https://api.coingecko.com/api/v3/coins/list')
    coins = pd.DataFrame(json.loads(response.text))

    return coins


def make_symbol_mapping(holders):

    coin_symbol = pd.Series(holders.columns.map(lambda x: x[1:].lower()), index=holders.columns)
    symbol_mapping = coin_symbol.rename_axis('w_symbol').rename('symbol').reset_index().set_index('symbol')
    return symbol_mapping


def selecting_list_of_tokens_to_request(coins, symbol_mapping):

    # selectig right coins to request from coingecko, removing some of them, because they
    # have similar symbols:
    drop = ['compound-coin', 'freetip', 'payperex', 'unicorn-token', 'universe-token']

    coins_indexes = coins[ ~coins.id.isin(drop) ].set_index('symbol').loc[symbol_mapping.index]

    # checking that there are no duplicated coin names and all selected correctly:
    assert not coins_indexes.index.duplicated().any()

    return coins_indexes


def augment_with_price_data_from_coingecko(coins_indexes, symbol_mapping):

    request_tokens = ','.join(coins_indexes.id.tolist()) + ',tezos'
    response = requests.get(f'https://api.coingecko.com/api/v3/simple/price?ids={request_tokens}&vs_currencies=eth')
    prices = pd.DataFrame(json.loads(response.text)).T
    tezos_prices = (prices / prices.loc['tezos']).eth.rename('tezos_price')
    coins_indexes = coins_indexes.join(tezos_prices, on='id')

    coins_indexes['w_index'] = coins_indexes.index.map(symbol_mapping['w_symbol'])
    coins_indexes = coins_indexes.reset_index().set_index('w_index')

    coins_prices = coins_indexes.rename(columns = lambda c: f'coingecko_{c}')
    return coins_prices


def collect_transaction_erc20_data_from_etherscan(address):

    url = f'https://api.etherscan.io/api?module=account&action=tokentx&address={address}&startblock=0&endblock=999999999&sort=asc'
    response = requests.get(url)
    ether_data = pd.DataFrame(json.loads(response.text)['result'])
    return ether_data


def process_etherscan_data(ether_data, address):

    ether_data['is_incoming'] = ether_data['to'] == address
    ether_data['is_outcoming'] = ether_data['from'] == address
    assert not any(ether_data['is_incoming'] & ether_data['is_outcoming'])
    assert all(ether_data['is_incoming'] | ether_data['is_outcoming'])

    ether_data['action'] = ether_data['is_incoming'].map({True: 'income', False: 'outcome'})
    ether_data['tokenDecimal'] = ether_data['tokenDecimal'].astype(int)
    ether_data['value_tokens'] = ether_data['value'].astype(float) / 10**ether_data['tokenDecimal']

    grouped_ehter_stats = ether_data.groupby(['tokenSymbol', 'action'])['value_tokens'].sum().unstack()
    grouped_ehter_stats['circulation'] = grouped_ehter_stats['income'] - grouped_ehter_stats['outcome']

    return grouped_ehter_stats


def change_index_to_wrapped(grouped_ehter_stats):

    index_mapping = list(grouped_ehter_stats.index)
    index_mapping = {symbol: 'w'+symbol for symbol in index_mapping}
    index_mapping['FTX Token'] = 'wFTT'
    grouped_ehter_stats.index = grouped_ehter_stats.index.map(index_mapping)

    grouped_ehter_stats = grouped_ehter_stats.rename(columns=lambda c: f'etherscan_{c}')

    return grouped_ehter_stats

### Collecting tokens & holders data from Tezos WRAP smart contract:

In [2]:
tezos_fa2_address = 'KT18fp5rcTW7mbWDmzFwjLDUhs5MeJmagDSZ'
ethereum_erc20_address = '0x5dc76fd132354be5567ad617fd1fe8fb79421d82'

tokens = response_tokens_deep(tezos_fa2_address, max_requests=20)
token_holders = collect_token_holders_data(tokens)
holders = token_holders.total.unstack().T

 15%|█▌        | 3/20 [00:05<00:30,  1.81s/it]
21it [00:38,  1.84s/it]


### Collecting price data from coingecko:

In [3]:
coins = request_coins_list_from_coingecko()
symbol_mapping = make_symbol_mapping(holders)
request_coins_indexes = selecting_list_of_tokens_to_request(coins, symbol_mapping)
coin_prices = augment_with_price_data_from_coingecko(request_coins_indexes, symbol_mapping)

### Collecting data from ehterscan API:

In [4]:
ether_stats = collect_transaction_erc20_data_from_etherscan(ethereum_erc20_address)
grouped_ether_stats = process_etherscan_data(ether_stats, ethereum_erc20_address)
grouped_ether_stats = change_index_to_wrapped(grouped_ether_stats)

### Combinig all data together:

In [5]:
tokens_info = tokens[['token_id', 'symbol', 'name', 'decimals']].set_index('symbol')

holders_info = pd.DataFrame({
    'holders count': holders.count(),
    'total wrapped': holders.sum()
})

df = pd.concat([tokens_info, holders_info, coin_prices, grouped_ether_stats], axis=1)
df['total_xtz_volume_wrapped'] = df['coingecko_tezos_price'] * df['total wrapped']
df = df.sort_values('total_xtz_volume_wrapped', ascending=False)

### Showing table with important stats:

In [6]:
columns_to_show = {
    'etherscan_income': 'Incoming tokens in Ethereum (etherscan data)',
    'etherscan_outcome': 'Outcoming tokens in Ethereum (ethrescan data)',
    'etherscan_circulation': 'Total ethereum wrapped token freezed on contract (etherscan data)',
    'holders count': 'Unique minters in Tezos (better-call.dev)',
    'total wrapped': 'Total minted Tokens in Tezos (better-call.dev)',
    'coingecko_tezos_price': 'Token Tezos price (coingecko)',
    'total_xtz_volume_wrapped': 'Total XTZ volume wrapped'
}

df[columns_to_show].rename(columns=columns_to_show).style.format('{:,.2f}')

,Incoming tokens in Ethereum (etherscan data),Outcoming tokens in Ethereum (ethrescan data),Total ethereum wrapped token freezed on contract (etherscan data),Unique minters in Tezos (better-call.dev),Total minted Tokens in Tezos (better-call.dev),Token Tezos price (coingecko),Total XTZ volume wrapped
wUNI,"24,983.85","16,257.06","8,726.79",15.00,"8,680.20",7.35,"63,784.55"
wMATIC,"228,911.44","4,063.06","224,848.38",10.00,"163,249.41",0.14,"22,239.67"
wWETH,164.61,81.63,82.97,17.00,31.78,493.00,"15,669.16"
wSUSHI,"7,413.04",99.30,"7,313.74",8.00,"4,878.03",2.65,"12,937.76"
wUSDT,"165,503.20","73,642.62","91,860.58",6.00,"59,611.34",0.19,"11,093.40"
wPAX,"285,872.39","231,652.52","54,219.87",4.00,"54,219.87",0.19,"10,096.73"
wUSDC,"213,038.32","71,355.75","141,682.57",27.00,"53,092.79",0.19,"9,912.40"
wLINK,"40,642.20","36,299.84","4,342.36",18.00,"1,383.28",6.84,"9,468.25"
wBUSD,"119,991.76","15,121.71","104,870.05",4.00,"45,405.88",0.19,"8,489.75"
wHUSD,"60,640.52",1.00,"60,639.52",4.00,"40,469.78",0.19,"7,544.76"


@ztepler